# Pré-processamento

O objetivo de pré-processar os dados é **aprimorar** o desempenho dos modelos de ML. Assim, um modelo treinado com dados pré-processados deve ter desempenho no mínimo equivalente a um modelo treinado com dados crus.

## Por que pré-processar os dados?

Dados no mundo real vêm

* incompletos

  ```person.occupation=''```


* ruidosos

  ```person.salary=-10; person.age=999```
  
  esses dados também podem ser chamados de *outliers*: são pontos que estão muito distantes da distribuição geral dos dados e em geral indicam algum erro na coleta ou no registro do dado


* inconsistentes

   ```personA.grade=B; personB.grade=8```
   
   pode acontecer ao integrar bases de dados diferentes
   

E os modelos de ML são sensíveis a tudo isso.

Além disso, lembrando que o problema de aprendizado é um problema de otimização, precisamos aplicar transormações nos dados de forma a facilitar a convergência dos otimizadores.


## As duas principais etapas de pré-processamento dos dados são

* limpeza: preenchimento de valores faltantes, remoção de *outliers*, resolução de inconsistências
* transformação: adequação dos dados para serem usados pelos modelos de ML; inclui normalização, agregação, redução e extração de *features*

Para a primeira etapa, limpeza, é necessário algum conhecimento sobre a natureza dos dados (por exemplo, saber que idade e data de nascimento são *features* equivalentes). Dependendo da natureza dos dados, é recomendável conversar com especialistas. Essa é uma etapa que parece simples mas costuma ser trabalhosa. Não deve ser menosprezada :)

Aqui, vamos usar um dataset já limpo e focar na tarefa de **transformar os dados** de forma que possam ser adequadamente interpretados pelos modelos.

## Nosso dataset

Vamos usar o dataset *Boston Housing Prices*, organizado pelo grupo de *Machine Learning* da *University of California Irvine* e disponível para baixar diretamente através do pacote `sklearn`.

Importando os pacotes que serão usados:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

Usamos a função `load_boston` do `sklearn` para carregar os dados e ver a descrição das variáveis, e depois o pacote `pandas` para facilitar a manipulação dos dados tabulares.

In [2]:
data = load_boston()

print(data.keys())
print(data.DESCR)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX

In [3]:
df = pd.DataFrame(data.data, columns=data.feature_names)
target = data.target

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


## Uma pipeline de pré-processamento de dados básica deve incluir
* tratamento de variáveis categóricas
* normalização/scaling
* seleção de *features*

### Tratamento de variáveis categóricas
Variáveis categóricas são variáveis discretizadas, que representam atributos qualitativos (por exemplo, raça) ou intervalos de atributos quantitativos (por exemplo, faixa etária).

Ao incluir essas variáveis num modelo de ML, é importante evitar introduzir relações de ordem inexistentes. Por exemplo, se há uma variável faixa etária que recebe 0 para [0-10], 1 para [11-20], 2 para [21-30], e assim por diante, é razoável que existam relações do tipo $0<1<2$ embutidas nos dados. Por outro lado, se há uma variável raça que recebe 0 para branco, 1 para amarelo, 2 para preto e 3 para outro, é recomendável criar $n$ novas variáveis binárias, uma para cada valor possível, a fim de evitar a introdução de relações do tipo branco > amarelo > preto. Este procedimento é chamado de **one-hot encoding**.

No nosso dataset, as seguintes variáveis são consideradas categóricas:
* CHAS: variável binária; não precisa de tratamento
* AGE: variável discretizada com 9 valores possíveis; podemos fazer one-hot encoding

In [4]:
df.CHAS.unique()

array([0., 1.])

In [5]:
df.RAD.unique()

array([ 1.,  2.,  3.,  5.,  4.,  8.,  6.,  7., 24.])

O one-hot encoding pode ser feito pelo `pandas`:

In [6]:
# one-hot encoding com pandas
rad_onehot = pd.get_dummies(df.RAD, prefix='RAD')
df = pd.concat([df, rad_onehot], axis=1)

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,LSTAT,RAD_1.0,RAD_2.0,RAD_3.0,RAD_4.0,RAD_5.0,RAD_6.0,RAD_7.0,RAD_8.0,RAD_24.0
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,...,4.98,1,0,0,0,0,0,0,0,0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,...,9.14,0,1,0,0,0,0,0,0,0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,...,4.03,0,1,0,0,0,0,0,0,0
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,...,2.94,0,0,1,0,0,0,0,0,0
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,...,5.33,0,0,1,0,0,0,0,0,0


### Normalização/scaling

É necessário para padronizar o valor das *features* quando elas estão em intervalos muito variados (por exemplo, idade e salário têm ordens de grandeza diferentes). Mantendo todas as *features* em intervalos de valores similares, a convergência dos modelos é acelerada e, em alguns casos, o desempenho final é melhorado.

As duas principais formas de se normalizar os dados são:

Min max scaling

$
\begin{align}
x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}
\end{align}
$


Standard scaling

$
\begin{align}
x_{scaled} = \frac{x - \mu}{\sigma}
\end{align}
$
  
Note que a normalização é feita para cada uma das *features*, ou seja, para cada coluna do vetor de *features* $X$.

O `sklearn` fornece funções para ambos os casos:

In [7]:
scaler = MinMaxScaler()
dff = df.copy()
dff[dff.columns] = scaler.fit_transform(df.values)
dff.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,LSTAT,RAD_1.0,RAD_2.0,RAD_3.0,RAD_4.0,RAD_5.0,RAD_6.0,RAD_7.0,RAD_8.0,RAD_24.0
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,...,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,0.040544,0.113636,0.391378,0.069170,0.349167,0.521869,0.676364,0.242381,0.371713,0.422208,...,0.301409,0.039526,0.047431,0.075099,0.217391,0.227273,0.051383,0.033597,0.047431,0.260870
std,0.096679,0.233225,0.251479,0.253994,0.238431,0.134627,0.289896,0.191482,0.378576,0.321636,...,0.197049,0.195035,0.212769,0.263812,0.412879,0.419485,0.220997,0.180367,0.212769,0.439543
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000851,0.000000,0.173387,0.000000,0.131687,0.445392,0.433831,0.088259,0.130435,0.175573,...,0.144040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.002812,0.000000,0.338343,0.000000,0.314815,0.507281,0.768280,0.188949,0.173913,0.272901,...,0.265728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.041258,0.125000,0.646628,0.000000,0.491770,0.586798,0.938980,0.369088,1.000000,0.914122,...,0.420116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
scaler = StandardScaler()
dff = df.copy()
dff[dff.columns] = scaler.fit_transform(df.values)
dff.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,LSTAT,RAD_1.0,RAD_2.0,RAD_3.0,RAD_4.0,RAD_5.0,RAD_6.0,RAD_7.0,RAD_8.0,RAD_24.0
count,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,...,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02
mean,-8.513173e-17,3.306534e-16,2.804081e-16,-3.100287e-16,-8.071058e-16,-5.189086e-17,-2.650493e-16,8.293761e-17,1.514379e-15,-9.934960e-16,...,-1.595123e-16,6.354162e-16,-1.068974e-15,5.487486e-16,-1.534126e-15,3.304340e-16,3.081637e-16,5.910950e-16,-5.813312e-16,3.427649e-15
std,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,...,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00
min,-4.197819e-01,-4.877224e-01,-1.557842e+00,-2.725986e-01,-1.465882e+00,-3.880249e+00,-2.335437e+00,-1.267069e+00,-9.828429e-01,-1.313990e+00,...,-1.531127e+00,-2.028602e-01,-2.231424e-01,-2.849501e-01,-5.270463e-01,-5.423261e-01,-2.327373e-01,-1.864533e-01,-2.231424e-01,-5.940885e-01
25%,-4.109696e-01,-4.877224e-01,-8.676906e-01,-2.725986e-01,-9.130288e-01,-5.686303e-01,-8.374480e-01,-8.056878e-01,-6.379618e-01,-7.675760e-01,...,-7.994200e-01,-2.028602e-01,-2.231424e-01,-2.849501e-01,-5.270463e-01,-5.423261e-01,-2.327373e-01,-1.864533e-01,-2.231424e-01,-5.940885e-01
50%,-3.906665e-01,-4.877224e-01,-2.110985e-01,-2.725986e-01,-1.442174e-01,-1.084655e-01,3.173816e-01,-2.793234e-01,-5.230014e-01,-4.646726e-01,...,-1.812536e-01,-2.028602e-01,-2.231424e-01,-2.849501e-01,-5.270463e-01,-5.423261e-01,-2.327373e-01,-1.864533e-01,-2.231424e-01,-5.940885e-01
75%,7.396560e-03,4.877224e-02,1.015999e+00,-2.725986e-01,5.986790e-01,4.827678e-01,9.067981e-01,6.623709e-01,1.661245e+00,1.530926e+00,...,6.030188e-01,-2.028602e-01,-2.231424e-01,-2.849501e-01,-5.270463e-01,-5.423261e-01,-2.327373e-01,-1.864533e-01,-2.231424e-01,1.683251e+00
max,9.933931e+00,3.804234e+00,2.422565e+00,3.668398e+00,2.732346e+00,3.555044e+00,1.117494e+00,3.960518e+00,1.661245e+00,1.798194e+00,...,3.548771e+00,4.929503e+00,4.481443e+00,3.509386e+00,1.897367e+00,1.843909e+00,4.296689e+00,5.363274e+00,4.481443e+00,1.683251e+00


### Seleção de features

Dado um dataset de *shape* $M x N$ (M amostras, N features), uma *rule of thumb* para um tamanho de dataset adequado é $M \geq 10N$. Nosso dataset tem shape 506x13, de um tamanho adequado, então não é preciso selecionar *features*.

Mas e se tivéssemos poucas amostras ou muitas *features*? Nesses casos, é necessário **extrair** ou **selecionar** *features* que contenham as informações mais relevantes.

Uma forma de fazer isso é avaliar as correlações entre *features* e *target* e descartar *features* sejam menos correlacionadas com o *target*. O método `SelectKBest` do `sklearn` faz isso.

In [9]:
fs = SelectKBest(score_func=f_regression, k=10)
X_selected = fs.fit_transform(dff, target)

cols = fs.get_support()
names = dff.columns.values[cols]
scores = fs.scores_[cols]
names_scores = list(zip(names, scores))

names_scores

[('CRIM', 89.48611475768118),
 ('INDUS', 153.95488313610983),
 ('NOX', 112.59148027969944),
 ('RM', 471.84673987638683),
 ('AGE', 83.47745921923611),
 ('RAD', 85.91427766984044),
 ('TAX', 141.76135657742293),
 ('PTRATIO', 175.10554287569468),
 ('LSTAT', 601.6178711098955),
 ('RAD_24.0', 93.90104712598881)]

## E se os dados não forem tabulares?

A extração de *features* é especialmente importante para dados com dimensão muito alta. Por exemplo, se tivermos imagens 32x32 e quisermos usar o valor de cada pixel como *feature*, teríamos 32x32 = 1024 *features*. Por isso, em campos como visão computacional e processamento de texto (NLP), são usados extratores de *features* especializados.

Alguns dos extratores de *features* tradicionais para imagens são:
* SURF
* SIFT

Alguns dos extratores de *features* tradicionais para texto são:
* bag of words
* TF-IDF

*Deep learning* é uma forma de se extrair *features* automaticamente e aprender uma tarefa ao mesmo tempo. Veremos um pouco sobre isso na última aula.